In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read.option("inferSchema", True).option("header", False).csv('covtype.data')
data.summary()


DataFrame[summary: string, _c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string, _c30: string, _c31: string, _c32: string, _c33: string, _c34: string, _c35: string, _c36: string, _c37: string, _c38: string, _c39: string, _c40: string, _c41: string, _c42: string, _c43: string, _c44: string, _c45: string, _c46: string, _c47: string, _c48: string, _c49: string, _c50: string, _c51: string, _c52: string, _c53: string, _c54: string]

In [4]:
data.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------

In [12]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, IntegerType
colnames=data.columns
data = data.toDF(*colnames)
data = data.withColumn("Cover_Type", col("Cover_Type").cast(DoubleType()))
for name in colnames[:-1]:
    data = data.withColumn(name, col(name).cast(IntegerType()))
data = data.na.drop()

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
train_data, test_data = data.randomSplit([0.9, 0.1])
train_data.cache()
test_data.cache()

input_cols = colnames[:-1]
vector_assembler = VectorAssembler(inputCols=input_cols, outputCol="featureVector")
assembled_train_data = vector_assembler.transform(train_data)

classifier = DecisionTreeClassifier(seed=1234, labelCol="Cover_Type", featuresCol="featureVector",
                                    predictionCol="prediction")
model = classifier.fit(assembled_train_data)

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(assembled_train_data)
predictions.select("Cover_Type", "prediction", "probability").show(10, truncate=False)

evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="prediction")

accuracy = evaluator.setMetricName("accuracy").evaluate(predictions)                           
print("Accuracy:", accuracy)

precision = evaluator.setMetricName("weightedPrecision").evaluate(predictions)
print("Precision:", precision)

recall = evaluator.setMetricName("weightedRecall").evaluate(predictions)
print("Recall:", recall)

+----------+----------+-----------------------------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                                        |
+----------+----------+-----------------------------------------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,3.168567807351077E-5,0.06660329531051964,0.6083333333333333,0.019613434727503168,0.001520912547528517,0.30389733840304184,0.0]|
|6.0       |4.0       |[0.0,0.0,0.03824362606232295,0.24291784702549576,0.6211048158640227,0.0084985835694051,0.08923512747875353,0.0]                    |
|6.0       |3.0       |[0.0,3.168567807351077E-5,0.06660329531051964,0.6083333333333333,0.019613434727503168,0.001520912547528517,0.30389733840304184,0.0]|
|6.0       |3.0       |[0.0,3.168567807351077E-5,0.0666032953105